# `Sign Language Prediction using ResNet`

In [ ]:
#imports
import os
import cv2
#import imghdr
#import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import load_model
from matplotlib import pyplot as plt
from keras.applications import ResNet50
from sklearn.preprocessing import normalize
from tensorflow.keras.preprocessing import image
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import math
from sklearn.model_selection import train_test_split
#----------------
import keras
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D, Dense,Flatten, Dropout
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.optimizers import SGD

In [ ]:
import cv2
import os
import numpy as np

data_path = 'Dataset\Signs'
high_level_directory = os.listdir(data_path)

# List to store image paths and corresponding labels
images = []
labels = []
classes=[]

for class_name in high_level_directory:
    classes.append(class_name)
    class_path = os.path.join(data_path, class_name)
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        img = cv2.imread(image_path)
        resized_img=cv2.resize(img,(224,224))
        images.append(resized_img)
        labels.append(class_name)


print(high_level_directory)

['bye', 'drink', 'eat', 'hello', 'help', 'love', 'morning', 'okay', 'please', 'stop', 'thankyou']


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels,stratify=labels, test_size=0.2)

In [ ]:
X_test[0].shape

(224, 224, 3)

In [ ]:
len(X_test)

189

In [ ]:
len(y_train)

753

In [ ]:
len(y_test)

189

## ResNet Model creation

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
IMG_SIZE = (224, 224)
train_dir = data_path

train_data, test_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE,
                                                                                validation_split=0.2,
                                                                                seed = 42,
                                                                                batch_size=32,
                                                                                subset="both")

Found 942 files belonging to 10 classes.
Using 754 files for training.
Using 188 files for validation.


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.optimizers.legacy import Adam


# Create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False # freeze base model layers

# Create Functional model
inputs = layers.Input(shape=input_shape, name="input_layer")
# Note: EfficientNetBX models have rescaling built-in but if your model didn't you could have a layer like below
# x = layers.Rescaling(1./255)(x)
x = base_model(inputs, training=False) # set base_model to inference mode only
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(classes))(x) # want one output neuron per class
# Separate activation of output layer so we can output float32 activations
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss="categorical_crossentropy", # Use sparse_categorical_crossentropy when labels are *not* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

16705208/16705208 [==============================] - 5s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 pooling_layer (GlobalAvera  (None, 1280)              0         
 gePooling2D)                                                    
                                                                 
 dense (Dense)               (None, 10)                12810     
                                                                 
 softmax_float32 (Activatio  (None, 10)                0         
 n)                                                              
                                                             

In [ ]:
# Turn off all warnings except for errors
tf.get_logger().setLevel('ERROR')

# Fit the model with callbacks
history_101_food_classes_feature_extract = model.fit(train_data,
                                                     epochs=20,
                                                     steps_per_epoch=len(train_data),
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)),)

Epoch 1/20
24/24 [==============================] - 29s 983ms/step - loss: 2.0960 - accuracy: 0.2546
Epoch 2/20
24/24 [==============================] - 28s 1s/step - loss: 1.5989 - accuracy: 0.6273
Epoch 3/20
24/24 [==============================] - 27s 1s/step - loss: 1.2790 - accuracy: 0.7427
Epoch 4/20
24/24 [==============================] - 25s 1s/step - loss: 1.0781 - accuracy: 0.7958
Epoch 5/20
24/24 [==============================] - 26s 1s/step - loss: 0.9250 - accuracy: 0.8448
Epoch 6/20
24/24 [==============================] - 25s 1000ms/step - loss: 0.8083 - accuracy: 0.8607
Epoch 7/20
24/24 [==============================] - 26s 1s/step - loss: 0.7246 - accuracy: 0.8727
Epoch 8/20
24/24 [==============================] - 21s 875ms/step - loss: 0.6487 - accuracy: 0.9125
Epoch 9/20
24/24 [==============================] - 21s 858ms/step - loss: 0.5838 - accuracy: 0.9271
Epoch 10/20
24/24 [==============================] - 21s 859ms/step - loss: 0.5361 - accuracy: 0.9324
Epo

In [ ]:
result_model = model.evaluate(test_data)
result_model

6/6 [==============================] - 7s 929ms/step - loss: 0.5880 - accuracy: 0.8351


[0.5879871845245361, 0.835106372833252]